In [35]:
import numpy as np
import pandas as pd

df = pd.read_csv('Shows.csv')
df.head()

,Unnamed: 0,ID,Name,Year,Rating,Duration,Reviews,Director,Genre,Link
0,0,tt5491994,Planet Earth II,2016,9.5,4h 58m,139,David Attenborough|Chadden Hunter|Elizabeth White,Documentary,https://www.imdb.com/title/tt5491994/
1,1,tt0795176,Planet Earth,2006,9.4,8h 58m,100,David Attenborough|Sigourney Weaver|Nikolay Dr...,Documentary,https://www.imdb.com/title/tt0795176/
2,2,tt0903747,Breaking Bad,2008,9.4,49m,4K,Bryan Cranston|Aaron Paul|Anna Gunn,Crime|Drama|Thriller,https://www.imdb.com/title/tt0903747/
3,3,tt0185906,Band of Brothers,2001,9.4,9h 54m,929,Scott Grimes|Damian Lewis|Ron Livingston,Drama|History|War,https://www.imdb.com/title/tt0185906/
4,4,tt7366338,Chernobyl,2019,9.3,5h 30m,3.3K,Jessie Buckley|Jared Harris|Stellan Skarsgård,Drama|History|Thriller,https://www.imdb.com/title/tt7366338/


#### 1.1) Removing Columns that are of no use

In [36]:
del df['Unnamed: 0']
del df['ID']
del df['Year']
del df['Rating']
del df['Duration']
del df['Reviews']
del df['Link']
del df['Genre']

df.head()

,Name,Director
0,Planet Earth II,David Attenborough|Chadden Hunter|Elizabeth White
1,Planet Earth,David Attenborough|Sigourney Weaver|Nikolay Dr...
2,Breaking Bad,Bryan Cranston|Aaron Paul|Anna Gunn
3,Band of Brothers,Scott Grimes|Damian Lewis|Ron Livingston
4,Chernobyl,Jessie Buckley|Jared Harris|Stellan Skarsgård


#### 1.2) Checking Null Values

In [37]:
df.isnull().sum()

Name        0
Director    0
dtype: int64

#### 1.3) Finding All Directors

In [38]:
directors = []
for i in df['Director']:
    
    directors += i.split('|')
    
directors = [i.strip() for i in directors]    
directors = list(set(directors))

len(directors)

669

#### 1.4) One Hot Conversion

In [39]:
one_hot = []

for i in df['Director']:
    
    i = [j.strip() for j in i.split('|')]        # Removing Spaces
    enc = [0 for i in range(len(directors))]
    
    for director in i:
        enc[directors.index(director)] = 1
    
    one_hot.append(enc)

#### 1.6) Converting OneHot to Array

In [40]:
one_hot = np.array(one_hot)

one_hot.shape

(250, 669)

#### 1.7) Adding OneHotData Data to DataFrame

In [41]:
for i in range(len(directors)):
    df[directors[i]] = one_hot[ : ,i]
    
df.head()

/var/folders/rq/lwddjvtn5n9gjw05hwrg3hp80000gn/T/ipykernel_27100/50977577.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[directors[i]] = one_hot[ : ,i]


,Name,Director,Hailee Steinfeld,Peter Dinklage,Iemasa Kayumi,Eugene Levy,Elisabeth Moss,Amy Poehler,Katee Sackhoff,Michael Rosenbaum,...,Max Mittelman,Jon Hamm,Rachel Brosnahan,Nikolay Drozdov,Susan Sarandon,Danny Pudi,Tamsin Greig,Michael Shannon,Salvatore Esposito,James May
0,Planet Earth II,David Attenborough|Chadden Hunter|Elizabeth White,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Planet Earth,David Attenborough|Sigourney Weaver|Nikolay Dr...,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,Breaking Bad,Bryan Cranston|Aaron Paul|Anna Gunn,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Band of Brothers,Scott Grimes|Damian Lewis|Ron Livingston,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Chernobyl,Jessie Buckley|Jared Harris|Stellan Skarsgård,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Prediction

#### 2.1) With Exact Directors

In [42]:
user = df[df['Name'] == 'Planet Earth'].values[0][2:]

len(user)

669

In [43]:
for i in df.values:
    if(list(user - i[2:]).count(0) == len(directors)):
        print(i[0])

Planet Earth


#### 2.2) With Better Reccomendation with Genre (Combinations)

In [44]:
def mod(lst):
    
    s = 0
    for i in lst:
        if (i < 0):
            s += (-i)
        else:
            s += (i)
    return s

In [45]:
user = df[df['Name'] == 'Planet Earth'].values[0][2:]
          
rel = []

for i in df.values:
    
    rel.append(mod(i[2:] - user))   
    
df['rel'] = rel

df.sort_values(by = 'rel').head(30)

,Name,Director,Hailee Steinfeld,Peter Dinklage,Iemasa Kayumi,Eugene Levy,Elisabeth Moss,Amy Poehler,Katee Sackhoff,Michael Rosenbaum,...,Jon Hamm,Rachel Brosnahan,Nikolay Drozdov,Susan Sarandon,Danny Pudi,Tamsin Greig,Michael Shannon,Salvatore Esposito,James May,rel
1,Planet Earth,David Attenborough|Sigourney Weaver|Nikolay Dr...,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
7,Our Planet,David Attenborough,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
23,The Blue Planet,David Attenborough|Pierce Brosnan,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
0,Planet Earth II,David Attenborough|Chadden Hunter|Elizabeth White,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
40,Africa,David Attenborough|Simon Blakeney|Mark Deeble,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
29,Frozen Planet,David Attenborough|Alec Baldwin|Chadden Hunter,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
17,Life,Oprah Winfrey|David Attenborough|Doug Allan,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
6,Blue Planet II,David Attenborough|Peter Drost|Roger Munns,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
25,Human Planet,John Hurt|Roger Munns,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
183,The IT Crowd,Chris O'Dowd|Richard Ayoade|Katherine Parkinson,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,6
